<a href="https://colab.research.google.com/github/kiplangatkorir/Multi-Agent-System/blob/main/multi_agent_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install transformers torch

In [6]:
# Load a larger language model from Hugging Face
generator = pipeline('text-generation', model='gpt2-large')

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [8]:

class LocalLLM:
    def generate(self, prompt):
        # Generate text using the local model
        response = generator(prompt, max_new_tokens=50, num_return_sequences=1)
        return response[0]['generated_text']

llm = LocalLLM()

class Agent:
    def __init__(self, name, instructions):
        self.name = name
        self.instructions = instructions

    def run(self, message):
        prompt = f"{self.instructions}\nTask: {message}"
        return llm.generate(prompt)

def debate(agents, topic):
    viewpoints = []
    for agent in agents:
        viewpoint = agent.run(f"Provide your perspective on: {topic}")
        viewpoints.append(f"{agent.name}: {viewpoint}")
    return viewpoints

def generate_explanation(process_log):
    explanation = "Problem-solving process:\n"
    for step in process_log:
        explanation += f"- {step['agent']} : {step['action']}\n"
    return explanation

# Create agents
coordinator = Agent("Coordinator", "You oversee the problem-solving process.")
math_expert = Agent("Math Expert", "You specialize in mathematics.")
fact_checker = Agent("Fact Checker", "You verify factual claims.")
creative_writer = Agent("Creative Writer", "You excel at creative writing.")

agents = [coordinator, math_expert, fact_checker, creative_writer]

def select_agents(task):
    relevant_agents = []
    if "math" in task.lower():
        relevant_agents.append(math_expert)
    if "fact" in task.lower() or "history" in task.lower():
        relevant_agents.append(fact_checker)
    if "story" in task.lower() or "creative" in task.lower():
        relevant_agents.append(creative_writer)
    return relevant_agents if relevant_agents else [random.choice(agents[1:])]

def execute_task(task):
    process_log = []

    # Coordinator breaks down the task
    subtasks_prompt = f"Break down this task into subtasks: {task}"
    subtasks_response = coordinator.run(subtasks_prompt)
    subtasks = subtasks_response.split('\n')[:3]  # Limit to 3 subtasks for clarity
    process_log.append({"agent": "Coordinator", "action": "Broke down the main task into subtasks"})

    results = []
    for subtask in subtasks:
        selected_agents = select_agents(subtask)

        if len(selected_agents) > 1:
            debate_results = debate(selected_agents, subtask)
            process_log.append({"agent": "Debate Module", "action": f"Conducted debate on {subtask}"})
            results.append(f"Debate on {subtask}:\n" + "\n".join(debate_results))
        else:
            agent_response = selected_agents[0].run(subtask)
            process_log.append({"agent": selected_agents[0].name, "action": f"Addressed subtask: {subtask}"})
            results.append(agent_response.strip())  # Ensure clean output

    # Coordinator synthesizes the results
    synthesis_prompt = "Synthesize the following points into a coherent story or response:\n" + "\n".join(results)
    final_result = coordinator.run(synthesis_prompt)
    process_log.append({"agent": "Coordinator", "action": "Synthesized final result"})

    explanation = generate_explanation(process_log)

    return final_result.strip(), explanation  # Clean and finalize the output

# Example usage
task = "Write a short story about a mathematician solving a historical mystery. Include accurate historical details and creative elements."
result, explanation = execute_task(task)

print("Final Result:")
print(result)
print("\nExplanation of the process:")
print(explanation)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Final Result:
You oversee the problem-solving process.
Task: Synthesize the following points into a coherent story or response:
You verify factual claims.
Task: You oversee the problem-solving process.
Tasks:
Identify:
- what are the potential risks associated with the change and its impact
- identify possible remedies and their impact or potential impacts
- identify how changes could be implemented to avert the risk from the change, and
Debate on Task: Break down this task into subtasks: Write a short story about a mathematician solving a historical mystery. Include accurate historical details and creative elements.:
Math Expert: You specialize in mathematics.
Task: Provide your perspective on: Task: Break down this task into subtasks: Write a short story about a mathematician solving a historical mystery. Include accurate historical details and creative elements.
The Story:
Two hundred years ago in 1539, an ancient woman by the name of Cazaly was sent into exile where she began work 